In [1]:
# loading the data
import pandas as pd

raw_data = pd.read_csv(r"C:\Users\rosel\Downloads\ETL_Midterm_Ziza_613\raw_data.csv")
incremental_data = pd.read_csv(r"C:\Users\rosel\Downloads\ETL_Midterm_Ziza_613\incremental_data.csv")


In [2]:
import os

# 1. Cleaning: Fill missing 'quantity' and 'unit_price' with 0, fill missing 'region' and 'customer_name' with 'Unknown'
def clean_data(df):
    df = df.copy()
    df['quantity'] = df['quantity'].fillna(0)
    df['unit_price'] = df['unit_price'].fillna(0)
    df['region'] = df['region'].fillna('Unknown')
    df['customer_name'] = df['customer_name'].fillna('Unknown')
    return df

# 2. Enrichment: Add 'total_price' column
def enrich_data(df):
    df = df.copy()
    df['total_price'] = df['quantity'] * df['unit_price']
    return df

# 3. Structural: Convert 'order_date' to datetime
def convert_dates(df):
    df = df.copy()
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
    return df

# 4. Categorization: Create 'order_size' category based on quantity
def categorize_order_size(df):
    df = df.copy()
    df['order_size'] = pd.cut(df['quantity'], bins=[-0.1, 0, 1, 3, float('inf')], labels=['None', 'Small', 'Medium', 'Large'])
    return df

def transform(df):
    df = clean_data(df)
    df = enrich_data(df)
    df = convert_dates(df)
    df = categorize_order_size(df)
    df = df.drop_duplicates()
    return df

# Apply transformations
transformed_full = transform(raw_data)
transformed_incremental = transform(incremental_data)

# Ensure output directory exists
os.makedirs('transformed', exist_ok=True)

# Save to CSV
transformed_full.to_csv('transformed/transformed_full.csv', index=False)
transformed_incremental.to_csv('transformed/transformed_incremental.csv', index=False)

In [7]:
# Show before and after for each transformation on incremental_data

# 1. Cleaning: Fill missing 'quantity' and 'unit_price' with 0, fill missing 'region' and 'customer_name' with 'Unknown'
print("Cleaning Step:")
print("Before:")
print(incremental_data[['quantity', 'unit_price', 'region', 'customer_name']])

cleaned = clean_data(incremental_data)
print("\nAfter:")
print(cleaned[['quantity', 'unit_price', 'region', 'customer_name']])
print("\nExplanation: Missing values in 'quantity' and 'unit_price' are filled with 0 to avoid calculation errors. Missing 'region' and 'customer_name' are filled with 'Unknown' for consistency.")



Cleaning Step:
Before:
   quantity  unit_price   region customer_name
0       NaN       900.0  Central         Alice
1       1.0       300.0  Central           NaN
2       1.0       600.0  Central           NaN
3       NaN       300.0  Central           NaN
4       2.0       600.0    North         Heidi
5       2.0       600.0  Central           NaN
6       1.0       600.0  Central           NaN
7       NaN       600.0      NaN           NaN
8       2.0       600.0  Central         Grace
9       NaN       900.0      NaN         Heidi

After:
   quantity  unit_price   region customer_name
0       0.0       900.0  Central         Alice
1       1.0       300.0  Central       Unknown
2       1.0       600.0  Central       Unknown
3       0.0       300.0  Central       Unknown
4       2.0       600.0    North         Heidi
5       2.0       600.0  Central       Unknown
6       1.0       600.0  Central       Unknown
7       0.0       600.0  Unknown       Unknown
8       2.0       600.0  Cent

In [ ]:
# 2. Structural: Convert 'order_date' to datetime
print("\nStructural Step:")
print("Before:")
print(enriched['order_date'].head(3))

converted = convert_dates(enriched)
print("\nAfter:")
print(converted['order_date'].head(3))
print("\nExplanation: 'order_date' is converted to datetime for proper date handling and analysis.")



Structural Step:
Before:
0    2024-05-09
1    2024-05-07
2    2024-05-04
Name: order_date, dtype: object

After:
0   2024-05-09
1   2024-05-07
2   2024-05-04
Name: order_date, dtype: datetime64[ns]

Explanation: 'order_date' is converted to datetime for proper date handling and analysis.


In [ ]:
# 3. Enrichment: Add 'total_price' column
print("\nEnrichment Step:")
print("Before:")
print(cleaned.head(3))

enriched = enrich_data(cleaned)
print("\nAfter:")
print(enriched.head(3))
print("\nExplanation: 'total_price' column is added as quantity * unit_price to provide order value.")


Enrichment Step:
Before:
   order_id customer_name product  quantity  unit_price  order_date   region
0       101         Alice  Laptop       0.0       900.0  2024-05-09  Central
1       102       Unknown  Laptop       1.0       300.0  2024-05-07  Central
2       103       Unknown  Laptop       1.0       600.0  2024-05-04  Central

After:
   order_id customer_name product  quantity  unit_price  order_date   region  \
0       101         Alice  Laptop       0.0       900.0  2024-05-09  Central   
1       102       Unknown  Laptop       1.0       300.0  2024-05-07  Central   
2       103       Unknown  Laptop       1.0       600.0  2024-05-04  Central   

   total_price  
0          0.0  
1        300.0  
2        600.0  

Explanation: 'total_price' column is added as quantity * unit_price to provide order value.


In [6]:
# 4. Categorization: Create 'order_size' category based on quantity
print("\nCategorization Step:")
print("Before:")
print(converted[['quantity']].head(5))

categorized = categorize_order_size(converted)
print("\nAfter:")
print(categorized[['quantity', 'order_size']].head(5))
print("\nExplanation: 'order_size' categorizes orders as None, Small, Medium, or Large based on quantity for easier analysis.")


Categorization Step:
Before:
   quantity
0       0.0
1       1.0
2       1.0
3       0.0
4       2.0

After:
   quantity order_size
0       0.0       None
1       1.0      Small
2       1.0      Small
3       0.0       None
4       2.0     Medium

Explanation: 'order_size' categorizes orders as None, Small, Medium, or Large based on quantity for easier analysis.
